# Differentiable LPC sanity check

In [1]:
import lpc_analysis
import IPython.display as ipd
import tensorflow as tf
import numpy as np

## Fetch sample audio

In [2]:
SR, X = lpc_analysis.fetch_audio("test_adv.wav")
ipd.display(ipd.Audio(X, rate = SR))

## LPC transform used in our defense (Non-differentiable)

In [3]:
w_coeffs, output_signal =  lpc_analysis.lpc_compress_decompress(X, SR, lpc_order = 20)
print("LPC coeffs")
print(w_coeffs)
print("Audio synthesized using original LPC transform in our defense.")
ipd.display(ipd.Audio(output_signal, rate = SR))

LPC coeffs
[[ 1.          0.59256124  0.12144794 ... -0.23750934 -0.25203743
  -0.03454165]
 [ 1.          0.73697098  0.23423864 ... -0.40101715 -0.28264124
  -0.14452808]
 [ 1.          0.57720455  0.43731282 ... -0.05795282  0.19023887
   0.21705574]
 ...
 [ 1.          0.15356776  0.01178765 ... -0.15965047 -0.23261942
  -0.13584416]
 [ 1.         -0.04709548  0.25824641 ...  0.04058795 -0.0436327
  -0.07255862]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
Audio synthesized using original LPC transform in our defense.


## Create differentiable Tensorflow graph for LPC

In [4]:
X_tf = tf.placeholder('float32', X.shape)
w_coeffs_tf, output_signal_tf = lpc_analysis.lpc_compress_decompress_tf(X_tf, SR, lpc_order = 20)

## Run the forward pass through the computational graph

In [5]:
sess = tf.InteractiveSession()
w_coeffs_tf_np, output_signal_tf_np = sess.run([w_coeffs_tf, output_signal_tf], feed_dict = {
    X_tf : X
})

In [6]:
print("LPC coeffs")
print(w_coeffs_tf_np)
print("Audio synthesized using LPC transform using Tensorflow")
ipd.display(ipd.Audio(output_signal_tf_np, rate = SR))

LPC coeffs
[[ 1.          0.5936817   0.12174708 ... -0.23868501 -0.2518406
  -0.03476696]
 [ 1.          0.7354033   0.23403193 ... -0.39907694 -0.28208745
  -0.1442753 ]
 [ 1.          0.57709944  0.4382914  ... -0.05742805  0.19108617
   0.21721138]
 ...
 [ 1.          0.15173171  0.01111978 ... -0.1593157  -0.23253584
  -0.1366495 ]
 [ 1.         -0.04657728  0.2609849  ...  0.04188117 -0.04205935
  -0.07082057]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
Audio synthesized using LPC transform using Tensorflow


## Check if the LPC coeffecients from numpy and tensorflow implementation are almost equal

In [7]:
def check_is_almost_equal(coeffs1, coeffs2, epsilon = 0.05):
    max_abs_diff = np.max(np.abs(coeffs1 - coeffs2))
    if max_abs_diff < epsilon:
        return True
    return False

print(check_is_almost_equal(w_coeffs, w_coeffs_tf_np))

True
